### Preliminary process

In [1]:
import os
import numpy as np

# 強制 Qt 在 Linux 環境下使用 X11
os.environ["QT_QPA_PLATFORM"] = "xcb"

# 防止 NumPy 佔用過多記憶體
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

# 固定隨機種子確保可重現性
np.random.seed(42)
os.environ["PYTHONHASHSEED"] = "42"

### Find cancer folder

In [2]:
# Import necessary module
import os

# Define the function
def find_folder(folder):
    """ Find folders for each cancer """
    # Create a empty list to store folder data
    cancer_folder = []
    
    # Walk across the file and folders in "folder"
    for dir in os.listdir(folder):
        cancer_folder.append(dir)
        
        # Remove .txt files in this list
        if dir.endswith(".txt"):
            cancer_folder.remove(dir)
    
    # Return this list for further utilization
    return cancer_folder
    

### Re-arrange the order of columns

In [3]:
import pyarrow.parquet as pq
import pyarrow as pa

# 第二步：統一欄位順序與缺值
def load_and_align(path_and_columns):
    path, all_columns = path_and_columns
    table = pq.read_table(path)
    table_columns = table.column_names
    missing_columns = [col for col in all_columns if col not in table_columns]

    # 加上缺失欄位（用 null）
    for col in missing_columns:
        table = table.append_column(col, pa.nulls(len(table)))

    # 重新排列欄位順序
    table = table.select(all_columns)
    return table

### Output expression data as parquet file

In [4]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import glob

# === 單個樣本的讀取與欄位補齊 ===
def load_sample(args):
    path, common_genes = args
    try:
        df = pd.read_csv(path, sep="\t", skiprows=6)
        df.columns = ["gene_id", "gene_name", "gene_type", "unstranded", "stranded_first",
                      "stranded_second", "tpm_unstranded", "fpkm_unstranded", "fpkm_uq_unstranded"]
        df["gene_id"] = df["gene_id"].str.rsplit(".", n=1).str[0]
        df = df.groupby("gene_id").first()
        series = df["tpm_unstranded"].reindex(common_genes)
        sample_name = os.path.basename(path).replace(".tsv", "")
        return sample_name, series
    except Exception:
        return None

# === 合併 parquet 子檔案 ===
def load_and_align(path_and_columns):
    path, all_columns = path_and_columns
    table = pq.read_table(path)
    table_columns = table.column_names
    missing_columns = [col for col in all_columns if col not in table_columns]
    for col in missing_columns:
        table = table.append_column(col, pa.nulls(len(table)))
    table = table.select(all_columns)
    return table

# === 主流程函數 ===
def preprocess(cancer_folder, input_folder, batch_size=50, n_jobs=8):
    # Step 1: 掃描檔案路徑
    tsv_paths = []
    for folder in cancer_folder:
        root_dir = os.path.join(input_folder, folder)
        for dirpath, _, filenames in os.walk(root_dir):
            for filename in filenames:
                if filename.endswith(".tsv"):
                    tsv_paths.append(os.path.abspath(os.path.join(dirpath, filename)))
    print(f"共找到 {len(tsv_paths)} 份 TSV 檔案。")

    # Step 2: 找出共通 gene_id
    common_genes = None
    for path in tqdm(tsv_paths, desc="計算共通基因中..."):
        df = pd.read_csv(path, sep="\t", skiprows=6, usecols=[0], names=["gene_id"], header=None)
        genes = set(gene.rsplit('.', 1)[0] for gene in df["gene_id"])
        common_genes = genes if common_genes is None else common_genes & genes
    common_genes = sorted(common_genes)

    # Step 3: 建立輸出目錄
    parquet_dir = os.path.join(input_folder, "final_data")
    os.makedirs(parquet_dir, exist_ok=True)

    # Step 4: 多核心平行讀取與批次寫入
    tasks = [(path, common_genes) for path in tsv_paths]
    n_jobs = n_jobs
    
    batch = []
    sample_names = []
    batch_id = 0

    with Pool(processes=n_jobs) as pool:
        for result in tqdm(pool.imap(load_sample, tasks), total=len(tasks), desc="處理並寫出 parquet 子檔案中..."):
            if result is None:
                continue
            sample_name, series = result
            batch.append(series)
            sample_names.append(sample_name)

            if len(batch) == batch_size:
                df_batch = pd.DataFrame(batch).T
                df_batch.columns = sample_names
                df_batch.index = common_genes
                table = pa.Table.from_pandas(df_batch)
                pq.write_table(table, os.path.join(parquet_dir, f"batch_{batch_id:05}.parquet"),
                               compression="snappy")
                batch = []
                sample_names = []
                batch_id += 1

    if batch:
        df_batch = pd.DataFrame(batch).T
        df_batch.columns = sample_names
        df_batch.index = common_genes
        table = pa.Table.from_pandas(df_batch)
        pq.write_table(table, os.path.join(parquet_dir, f"batch_{batch_id:05}.parquet"),
                       compression="snappy")

    # Step 5: 分批合併所有 parquet 子檔案為 final_data.parquet
    print("分批合併 parquet 子檔案中...")
    paths = sorted(glob.glob(os.path.join(parquet_dir, "*.parquet")))

    # 讀入每個 parquet 檔為 pandas DataFrame，並以 axis=1 合併（橫向）
    df_parts = []
    for path in tqdm(paths, desc="讀入 parquet 子檔"):
        df = pd.read_parquet(path)  # 應為 gene_id 為 index、樣本為欄位
        df_parts.append(df)

    # 最終橫向合併（以 gene 為索引）
    final_df = pd.concat(df_parts, axis=1)

    # 儲存為 Parquet
    final_path = os.path.join(input_folder, "final_data.parquet")
    final_df.to_parquet(final_path, compression="snappy")
    print(f"✅ 最終資料完成：{final_path}")


### Pybiomart to find specific genes

In [5]:
# Import necessary modules
from pybiomart import Server

# Define the function
def load_centrosome(do_centro = True):
    if do_centro == True:
        print(" [i] Accessing centrosome genes...")
        """Establish a dictionary of centrosome protein coding genes"""
        # Create connection
        server = Server(host = "http://www.ensembl.org/")
        dataset = server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl']
    
        # Debug: Check the columns of "dataset"
        # col = dataset.attributes
        # print(col)
    
        # Get ensembl_gene_id, go_id, and gene_biotype
        genes_df = dataset.query(attributes = ["ensembl_gene_id", "go_id", "gene_biotype", 'hgnc_symbol'])
    
        # Pick genes related to centrosome protein coding
        genes_df = genes_df[genes_df["GO term accession"] == "GO:0005813"]
        genes_df = genes_df[genes_df["Gene type"] == "protein_coding"]
    
        return genes_df["Gene stable ID"].tolist()
    
    else:
        return []

### Load parquet file

In [6]:
# Import necessary modules
import polars as pl

# Define the function
def load_parquet(parquet_input, genes_list, index_col: str = None):
    """Read in the parquet file and prepare it for t-SNE"""
    
    print(" [i] Loading TCGA parquet data...")
    # Read the full parquet file with lazy loading
    df = pl.read_parquet(parquet_input).lazy()
    
    # Select f64 columns and convert them to f32 to save memory
    schema = df.collect_schema()
    convert = [
        pl.col(name).cast(pl.Float32).alias(name)
        if dtype == pl.Float64 else pl.col(name)
        for name, dtype in schema.items()
    ]
    
    # Apply the convertion to dataframe w/o materializing
    converted = df.select(convert)

    # Read the full dataframe
    df = converted.collect()

    if index_col is None:
        for candidate in ["__index_level_0__", "index", "Unnamed: 0"]:
            if candidate in df.columns:
                index_col = candidate
                break
    
    # Move index column to the first column
    if df.columns[0] != index_col:
        df = df.select([index_col] + [col for col in df.columns if col != index_col])
    
    if genes_list != []:
        gene_id_col = df.columns[0]
        genes = genes_list
        df = df.filter(pl.col(gene_id_col).is_in(genes))
        index_values = df[gene_id_col].to_list()
        df_no_index = df.drop(gene_id_col)
    else:
        gene_id_col = df.columns[0]
        df = df
        index_values = df[gene_id_col].to_list()
        df_no_index = df.drop(gene_id_col)
    
    # Set teh first column to rownames and transpose DataFrame
    df_transposed = df_no_index.transpose(include_header = True,
                                          column_names = index_values)
    
    # Turn the index back to column names
    df_transposed = df_transposed.with_columns(
        pl.Series(name = "index", values = df_no_index.columns)
    )

    return df_transposed
            
                

### Load and align metadata labels

In [7]:
import pandas as pd
import polars as pl
import csv

def load_metadata_labels(metadata_path, df_transposed):
    print(" [i] Reading sample metadata...")
    metadata_df = pd.read_csv(metadata_path, delimiter = ",", quoting = csv.QUOTE_ALL, escapechar='\\')
    metadata_df.set_index("filename", inplace = True)
    
    # Debug: Polars DataFrame
    if not isinstance(df_transposed, pl.DataFrame):
        raise ValueError(" [!] df_transposed 必須為 Polars 資料框")
    
    # Turn the 0 column (sample name) to index
    index_series = df_transposed.select(pl.col(df_transposed.columns[0])).to_series()
    df_transposed = df_transposed.drop(df_transposed.columns[0])
    df_transposed = df_transposed.with_columns(pl.Series(name="index", values=index_series))

    df_transposed = df_transposed.to_pandas(use_pyarrow_extension_array=False)
    df_transposed.set_index("index", inplace=True)

    # Align process
    index_list = df_transposed.index
    matched_index = metadata_df.index.intersection(index_list)
    metadata_matched = metadata_df.loc[matched_index]
    
    # Debug: Sample alingment
    # print(df_transposed.index[:5])
    # print(matched_index[:5])

    # Debug: Missing values
    # missing = [x for x in matched_index if x not in df_transposed.index]
    # print(f"[!] 找不到的樣本數: {len(missing)}")
    # print(missing[:5])
    
    # Choose corresponding rows
    print(" [i] Choosing corresponding rows...")
    df_matched = df_transposed.loc[matched_index, :]
    
    # Ensure fully aligned
    print(" [!] 是否完全對齊？", all(metadata_matched.index == df_matched.index))
    print(" [!] 樣本數：", len(metadata_matched))
    
    # Return df_matched and metadata_matched
    return df_matched, metadata_matched

### Pre-process (excluding NaN and standarize)

In [8]:
# 預處理數據
def data_process(df_matched):
    print(" [i] Pre-processing data...")
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    df_matched = df_matched.select_dtypes(include=[np.number])  # 只保留數值欄位
    df_matched = df_matched.loc[:, df_matched.notna().any()] # Excluding NaN
    data_scaled = scaler.fit_transform(df_matched)
    return data_scaled, data_scaled.shape

### t-SNE Process

In [9]:
# t-SNE 降維
def perform_tsne_np(data_scaled, perplexity = 30, learning_rate = 200, 
                    n_components=2, max_iter=2000, init = "pca", random_state=42):
    print(" [i] Performing t-SNE_NP...")
    from sklearn.manifold import TSNE
    tsne = TSNE(n_components=n_components, perplexity=perplexity, 
                learning_rate=learning_rate, max_iter=max_iter, 
                init = init, random_state = random_state)
    
    return tsne.fit_transform(data_scaled)

### Plotting t-SNE image

In [10]:
import plotly.express as px

# 繪製 t-SNE 圖形
def plot_tsne(tsne_results, label, output_path):
    print(" [i] Drawing t-SNE plot...")
    import seaborn as sns
    tissues = label.unique()
    palette = sns.color_palette("hls", len(tissues))
    custom_order = (sorted(set(tissues)))
    color_map = {
        tissue: f"rgb({int(r*255)}, {int(g*255)}, {int(b*255)})"
        for tissue, (r, g, b) in zip(custom_order, palette)
    }
    fig = px.scatter(x = tsne_results[:, 0], y = tsne_results[:, 1], 
                     color = label, color_discrete_map = color_map,
                     hover_name = label, 
               title = "t-SNE visualization of TCGA Data", template='plotly_white', 
               width = 2000, height = 1600, 
               labels = {
                    'tSNE-1': 't-SNE dimension 1',
                    'tSNE-2': 't-SNE dimension 2'
               }
            )
    fig.update_layout(
        legend_title_text="Tissues",
        legend=dict(
            x=1,
            y=1,
            xanchor='left',
            yanchor='top'
        ),
        margin=dict(t=50, b=50, l=50, r=50)  # 避免圖例或標題被裁切
    )
    fig.show() # 如果要顯示圖片則取消註解
    fig.write_image(output_path)
    print(f" [i] t-SNE figure is saved as: {output_path}")

### Main Process

In [11]:

if __name__ == '__main__':
    
    import time
    # Time start
    start = time.time()
    
    # Set constant variables
    input_folder = r"/home/terry-liu/桌面/tcga_cancer"
    parquet_input = r"/home/terry-liu/桌面/tcga_cancer/final_data.parquet"
    metadata_path = r"/home/terry-liu/桌面/tcga_cancer/tcga_metadata.csv"
    output_path = r"/home/terry-liu/桌面/tsne_plot"
    
    if not os.path.isfile(parquet_input):
        # Find folders for each cancer
        print("Find folders for each cancer...")
        cancer_folder = find_folder(input_folder)
    
        print("Combining the tsv files to a intact dataframe...")
        preprocess(cancer_folder, input_folder)
    
    # genes_list = load_centrosome(do_centro = True) # Set True to enable centrosome gene filtering
    genes_list = []
    df_transposed = load_parquet(parquet_input, genes_list)
    df_matched, metadata_matched = load_metadata_labels(metadata_path, df_transposed)
    
    # Label for t-SNE plot
    label = metadata_matched["cancer_type"]
    
    # Pre-processing of data
    data_scaled, shape = data_process(df_matched)
    
    # t-SNE process
    tsne_results = perform_tsne_np(data_scaled)
    
    # Plotting
    output_image = os.path.join(output_path, "tcga_tsne_all_gene.png")
    plot_tsne(tsne_results, label, output_image)
    
    # End
    print(" [i] t-SNE analysis is completed.")

    end = time.time()
    elapsed_time = int(end - start)
    hours = elapsed_time // 3600
    minutes = elapsed_time % 3600 // 60
    seconds = elapsed_time % 60

    print(f" [i] Time spent: {hours} hours, {minutes} minutes and {seconds} seconds.")
    

 [i] Loading TCGA parquet data...
 [i] Reading sample metadata...
 [i] Choosing corresponding rows...
 [!] 是否完全對齊？ True
 [!] 樣本數： 10759
 [i] Pre-processing data...
 [i] Performing t-SNE_NP...
 [i] Drawing t-SNE plot...


 [i] t-SNE figure is saved as: /home/terry-liu/桌面/tsne_plot/tcga_tsne_all_gene.png
 [i] t-SNE analysis is completed.
 [i] Time spent: 0 hours, 4 minutes and 43 seconds.
